In [ ]:
!pip install torch_summary

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import resnest.torch as resnest_torch
import typing as tp
from torchsummary import summary


In [2]:
# def get_model():
#     model =getattr(resnest_torch, 'resnest50_fast_1s1x64d')(pretrained=False)
#     del model.fc
#     # # use the same head as the baseline notebook.
#     model.fc = nn.Sequential(
#         nn.Linear(2048, 1024), nn.ReLU(), nn.Dropout(p=0.2),
#         nn.Linear(1024, 1024), nn.ReLU(), nn.Dropout(p=0.2),
#         nn.Linear(1024, 264))
    
#     return model

In [10]:
def get_model():
    model =getattr(resnest_torch, 'resnest50_fast_1s1x64d')(pretrained=True)
    del model.fc
    # # use the same head as the baseline notebook.
    model.fc = nn.Sequential(
        nn.Linear(2048, 1024), nn.ReLU(), nn.Dropout(p=0.2),
        nn.Linear(1024, 1024), nn.ReLU(), nn.Dropout(p=0.2),
        nn.Linear(1024, 264))
    
    conv1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
                 nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                 nn.ReLU(inplace=True),
                 nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
                 nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                 nn.ReLU(inplace=True),
                 nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False))    
    
    layers = list(model.children())[1:]
#         layers.append(nn.AdaptiveMaxPool2d(1))
    model = nn.Sequential(conv1,*layers)
    
    return model

In [11]:
model = get_model()
summary(model,(1,224,224)) # summary(model,(channels,H,W))


Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 112, 112]        --
|    └─Conv2d: 2-1                       [-1, 32, 112, 112]        288
|    └─BatchNorm2d: 2-2                  [-1, 32, 112, 112]        64
|    └─ReLU: 2-3                         [-1, 32, 112, 112]        --
|    └─Conv2d: 2-4                       [-1, 32, 112, 112]        9,216
|    └─BatchNorm2d: 2-5                  [-1, 32, 112, 112]        64
|    └─ReLU: 2-6                         [-1, 32, 112, 112]        --
|    └─Conv2d: 2-7                       [-1, 64, 112, 112]        18,432
├─BatchNorm2d: 1-2                       [-1, 64, 112, 112]        128
├─ReLU: 1-3                              [-1, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [-1, 64, 56, 56]          --
├─Sequential: 1-5                        [-1, 256, 56, 56]         --
|    └─Bottleneck: 2-8                   [-1, 256, 56, 56]         --
|    |

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 112, 112]        --
|    └─Conv2d: 2-1                       [-1, 32, 112, 112]        288
|    └─BatchNorm2d: 2-2                  [-1, 32, 112, 112]        64
|    └─ReLU: 2-3                         [-1, 32, 112, 112]        --
|    └─Conv2d: 2-4                       [-1, 32, 112, 112]        9,216
|    └─BatchNorm2d: 2-5                  [-1, 32, 112, 112]        64
|    └─ReLU: 2-6                         [-1, 32, 112, 112]        --
|    └─Conv2d: 2-7                       [-1, 64, 112, 112]        18,432
├─BatchNorm2d: 1-2                       [-1, 64, 112, 112]        128
├─ReLU: 1-3                              [-1, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [-1, 64, 56, 56]          --
├─Sequential: 1-5                        [-1, 256, 56, 56]         --
|    └─Bottleneck: 2-8                   [-1, 256, 56, 56]         --
|    |

In [3]:
from resnet import ResNet, Bottleneck

In [8]:
def resnest50(pretrained=False, root='~/.encoding/models', **kwargs):
    model = ResNet(Bottleneck, [3, 4, 6, 3],
                   radix=2, groups=1, bottleneck_width=64,
                   deep_stem=True, stem_width=32, avg_down=True,
                   avd=True, avd_first=False, **kwargs)
    if pretrained:
        model.load_state_dict(torch.hub.load_state_dict_from_url(
            resnest_model_urls['resnest50'], progress=True, check_hash=True))
        
    del model.fc
    # # use the same head as the baseline notebook.
    model.fc = nn.Sequential(
        nn.Linear(2048, 1024), nn.ReLU(), nn.Dropout(p=0.2),
        nn.Linear(1024, 1024), nn.ReLU(), nn.Dropout(p=0.2),
        nn.Linear(1024, 264))
    return model

In [9]:
model = resnest50()

In [10]:
summary(model,(1,224,224)) # summary(model,(channels,H,W))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 112, 112]        --
|    └─Conv2d: 2-1                       [-1, 32, 112, 112]        288
|    └─BatchNorm2d: 2-2                  [-1, 32, 112, 112]        64
|    └─ReLU: 2-3                         [-1, 32, 112, 112]        --
|    └─Conv2d: 2-4                       [-1, 32, 112, 112]        9,216
|    └─BatchNorm2d: 2-5                  [-1, 32, 112, 112]        64
|    └─ReLU: 2-6                         [-1, 32, 112, 112]        --
|    └─Conv2d: 2-7                       [-1, 64, 112, 112]        18,432
├─BatchNorm2d: 1-2                       [-1, 64, 112, 112]        128
├─ReLU: 1-3                              [-1, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [-1, 64, 56, 56]          --
├─Sequential: 1-5                        [-1, 256, 56, 56]         --
|    └─Bottleneck: 2-8                   [-1, 256, 56, 56]         --
|    |

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 112, 112]        --
|    └─Conv2d: 2-1                       [-1, 32, 112, 112]        288
|    └─BatchNorm2d: 2-2                  [-1, 32, 112, 112]        64
|    └─ReLU: 2-3                         [-1, 32, 112, 112]        --
|    └─Conv2d: 2-4                       [-1, 32, 112, 112]        9,216
|    └─BatchNorm2d: 2-5                  [-1, 32, 112, 112]        64
|    └─ReLU: 2-6                         [-1, 32, 112, 112]        --
|    └─Conv2d: 2-7                       [-1, 64, 112, 112]        18,432
├─BatchNorm2d: 1-2                       [-1, 64, 112, 112]        128
├─ReLU: 1-3                              [-1, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [-1, 64, 56, 56]          --
├─Sequential: 1-5                        [-1, 256, 56, 56]         --
|    └─Bottleneck: 2-8                   [-1, 256, 56, 56]         --
|    |

In [11]:
list(model.children())[0]

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)

In [14]:
nn.Sequential = (nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
nn.ReLU(inplace=True),
nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
nn.ReLU(inplace=True),
nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False))